# Climatologies and Anomalies

This notebook will demonstrate how to use the earthkit libraries to access some ERA5 data, calculate some climatologies and anomalies then plot the output.

For this exercise we will use the earthkit-data package to access the data, earthkit-climate to calculate our climatologies and anomalies and earthkit-maps to plot the results.

In [1]:
from earthkit import data as ek_data
from earthkit import climate as ek_climate
from earthkit import maps as ek_maps

In [4]:
cds_dataset_name = 'reanalysis-era5-single-levels'

# We use an Earthkit bounding box object to describe our area, this clears up any lack of clarity of the order of
#  North, South, East, West.

area = ek_data.utils.bbox.BoundingBox(north=80, south=30, west=-30, east=60)
cds_request = {
    'product_type': 'reanalysis',
    'variable': '2m_temperature',
    'year': ["2015", "2016", "2017"],
    'month': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12',
    ],
    'day': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12',
        '13', '14', '15',
        '16', '17', '18',
        '19', '20', '21',
        '22', '23', '24',
        '25', '26', '27',
        '28', '29', '30',
        '31',
    ],
    'time': [
        '00:00', '06:00', '12:00', '18:00',
    ],
    'area': area
}
era5_data = ek_data.from_source("cds", cds_dataset_name, cds_request)


2023-08-31 09:59:51,423 INFO Welcome to the CDS
2023-08-31 09:59:51,423 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-31 09:59:51,491 INFO Request is queued
2023-08-31 09:59:52,532 INFO Request is running
2023-08-31 10:04:09,633 INFO Request is completed
2023-08-31 10:04:09,634 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data6/adaptor.mars.internal-1693472391.9870067-9401-10-3006dbbf-bbe0-4994-9236-0da849051100.grib to /var/folders/l2/529q7bzs665bnrn7_wjx1nsr0000gn/T/earthkit-data-edwardcomyn-platt/cds-retriever-5da7848bb093a3428d7d06865db88d2bceac0fbe5ff7bc71ddf2a2fb50d9432a.cache.tmp (607.6M)
2023-08-31 10:05:37,581 INFO Download rate 6.9M/s  


In [ ]:
climatology = ek_climate.climatology.mean(era5_data, frequency='month')

ValueError: Invalid frequency 'month' - see xarray documentation for a full list of valid frequencies.